In [0]:
# 📦 GOLD LAYER: Pedidos Pago
from pyspark.sql.functions import col, monotonically_increasing_id, coalesce, lit, upper, trim

# 🔄 Eliminar tablas si ya existen
drop_list = [
    "default.dim_proveedor",
    "default.dim_area",
    "default.dim_pedido",
    "default.fact_pedidos_pago"
]
for table in drop_list:
    spark.sql(f"DROP TABLE IF EXISTS {table}")

# 1️⃣ Leer la tabla Silver
silver_df = spark.read.table("default.silver_pedidos_pago")

# ✅ Normalizar columnas clave para evitar nulos
silver_df = silver_df.withColumn("area", upper(trim(coalesce(col("area"), lit("SIN_AREA"))))) \
                     .withColumn("area_solicitante", upper(trim(coalesce(col("area_solicitante"), lit("SIN_AREA")))))

# 2️⃣ Crear DIMENSIONES

## 🎯 Dimensión Proveedor
dim_proveedor = silver_df.select("codigo_proveedor", "nombre_proveedor").dropDuplicates()
dim_proveedor = dim_proveedor.withColumn("id_proveedor", monotonically_increasing_id())
dim_proveedor.write.format("delta").mode("overwrite").saveAsTable("default.dim_proveedor")

## 🎯 Dimensión Área
dim_area = silver_df.select("area", "area_solicitante").dropDuplicates()
dim_area = dim_area.withColumn("id_area", monotonically_increasing_id())
dim_area.write.format("delta").mode("overwrite").saveAsTable("default.dim_area")

## 🎯 Dimensión Pedido (solo con pedido_compra y estado_pedido)
dim_pedido = silver_df.select("pedido_compra", "estado_pedido").dropDuplicates()
dim_pedido = dim_pedido.withColumn("id_pedido", monotonically_increasing_id())
dim_pedido.write.format("delta").mode("overwrite").saveAsTable("default.dim_pedido")

# 3️⃣ Crear tabla de HECHOS

## 🧱 Agregar ID de dimensiones
fact_df = silver_df \
    .join(dim_proveedor, ["codigo_proveedor", "nombre_proveedor"], "left") \
    .join(dim_area, ["area", "area_solicitante"], "left") \
    .join(dim_pedido, ["pedido_compra", "estado_pedido"], "left")

## 🧱 Seleccionar columnas finales
fact_pedidos_pago = fact_df.select(
    "id_proveedor", "id_area", "id_pedido",
    "total_pago_soles",
    "fecha_recepcion", "fecha_generacion", "fecha_entrega_servicio",
    "fecha_documento", "fecha_aceptacion", "fecha_envio_proveedor",
    "fecha_mesa_partes", "fecha_contabilidad", "fecha_pago", "fecha_registro_conta"
).withColumn("id_fact_pedido_pago", monotonically_increasing_id())

# 4️⃣ Guardar tabla de hechos
fact_pedidos_pago.write.format("delta").mode("overwrite").saveAsTable("default.fact_pedidos_pago")

print("✅ Tablas GOLD creadas exitosamente en el schema 'default': fact_pedidos_pago + dimensiones")


In [0]:
%sql
select id_proveedor,codigo_proveedor,nombre_proveedor from default.dim_proveedor order by id_proveedor

In [0]:
%sql
SELECT id_area, area, area_solicitante
FROM default.dim_area
ORDER BY id_area

In [0]:
%sql
SELECT id_pedido, pedido_compra, estado_pedido
FROM default.dim_pedido
ORDER BY id_pedido

In [0]:
%sql
SELECT * FROM default.dim_pedido WHERE pedido_compra IS NULL OR estado_pedido IS NULL OR codigo_solicitud IS NULL

In [0]:
%sql
SELECT
  id_fact_pedido_pago,
  id_proveedor,
  id_area,
  id_pedido,
  total_pago_soles,
  fecha_recepcion,
  fecha_generacion,
  fecha_entrega_servicio,
  fecha_documento,
  fecha_aceptacion,
  fecha_envio_proveedor,
  fecha_mesa_partes,
  fecha_contabilidad,
  fecha_pago,
  fecha_registro_conta
FROM default.fact_pedidos_pago
order by id_fact_pedido_pago